# Graded Lab: Agentic Workflows

In this lab, you will build an agentic system that generates a short research report through planning, external tool usage, and feedback integration. Your workflow will involve:

### Agents

* **Planning Agent / Writer**: Creates an outline and coordinates tasks.
* **Research Agent**: Gathers external information using tools like Arxiv, Tavily, and Wikipedia.
* **Editor Agent**: Reflects on the report and provides suggestions for improvement.

### Research Tools

By importing `research_tools`, you gain access to several search utilities:

- `research_tools.arxiv_search_tool(query)` → search academic papers from **arXiv**  

  *Example:* `research_tools.arxiv_search_tool("neural networks for climate modeling")`

- `research_tools.tavily_search_tool(query)` → perform web searches with the **Tavily API**  

  *Example:* `research_tools.tavily_search_tool("latest trends in sunglasses fashion")`

- `research_tools.wikipedia_search_tool(query)` → retrieve summaries from **Wikipedia**  

  *Example:* `research_tools.wikipedia_search_tool("Ensemble Kalman Filter")`

Run the cell below to make them available.

In [ ]:
# =========================
# Imports
# =========================

# --- Standard library 
from datetime import datetime
import re
import json
import ast


# --- Third-party ---
from IPython.display import Markdown, display
from aisuite import Client

# --- Local / project ---
from helpers import research_tools
from helpers import unittests

### Initialize client

Create a shared client instance for upcoming calls.

In [ ]:
CLIENT = Client()

## Exercise 1: planner_agent

### Objective
Correctly set up a call to a language model (LLM) to generate a research plan.

### Instructions

1. **Focus Areas**:
   - Ensure `CLIENT.chat.completions.create` is correctly configured.
   - Pass the `model` and `messages` parameters correctly:
     - **Model**: Use `"openai:o4-mini"` by default.
     - **Messages**: Set with `{"role": "user", "content": user_prompt}`.
     - **Temperature**: Fixed at 1 for creative outputs.

In [ ]:
# GRADED FUNCTION: planner_agent

def planner_agent(topic: str, model: str = "openai:o4-mini") -> list[str]:
    """
    Generates a plan as a Python list of steps (strings) for a research workflow.

    Args:
        topic (str): Research topic to investigate.
        model (str): Language model to use.

    Returns:
        List[str]: A list of executable step strings.
    """

    
    # Build the user prompt
    user_prompt = f"""
    You are a planning agent responsible for organizing a research workflow with multiple intelligent agents.

    🧠 Available agents:
    - A research agent who can search the web, Wikipedia, and arXiv.
    - A writer agent who can draft research summaries.
    - An editor agent who can reflect and revise the drafts.

    🎯 Your job is to write a clear, step-by-step research plan **as a valid Python list**, where each step is a string.
    Each step should be atomic, executable, and must rely only on the capabilities of the above agents.

    🚫 DO NOT include irrelevant tasks like "create CSV", "set up a repo", "install packages", etc.
    ✅ DO include real research-related tasks (e.g., search, summarize, draft, revise).
    ✅ DO assume tool use is available.
    ✅ DO NOT include explanation text — return ONLY the Python list.
    ✅ The final step should be to generate a Markdown document containing the complete research report.

    Topic: "{topic}"
    """

    # Add the user prompt to the messages list
    messages = [{"role": "user", "content": user_prompt}]

    ### START CODE HERE ###

    # Call the LLM
    response = CLIENT.chat.completions.create( 
        model=model,
        messages=messages,
        temperature=1, 
    )

    ### END CODE HERE ###

    # Extract message from response
    steps_str = response.choices[0].message.content.strip()

    # Parse steps
    steps = ast.literal_eval(steps_str)

    return steps

In [ ]:
# Test your code!
unittests.test_planner_agent(planner_agent)

## Exercise 2: research_agent

### Objective
Set up a call to a language model (LLM) to perform a research task using various tools.

### Instructions

**Focus Areas**:

- **Creating a Custom Prompt**:
  - **Define the Role**: Clearly specify the role, such as "research assistant."
  - **List Available Tools** (as strings inside the prompt, not the actual functions):
    - Use `arxiv_tool` to find academic papers.
    - Use `tavily_tool` for general web searches.
    - Use `wikipedia_tool` for accessing encyclopedic knowledge.
  - **Specify the Task**: Include a placeholder in your prompt for defining the specific task that needs to be accomplished.
  - **Include Date Information**: Add a placeholder for the current date or time to provide context.

- **Creating Messages Dict**:
  - Ensure the `messages` are correctly set with `{"role": "user", "content": prompt}`.

- **Creating Tools List**:
  - Create a list of tools for use, such as `research_tools.arxiv_search_tool`, `research_tools.tavily_search_tool`, and `research_tools.wikipedia_search_tool`.

- **Correctly Setting the Call to the LLM**:
  - Pass the `model`, `messages`, and `tools` parameters accurately.
  - Set `tool_choice` to `"auto"` for automatic tool selection.
  - Limit interactions with `max_turns=6`.

In [ ]:
# GRADED FUNCTION: research_agent

def research_agent(task: str, model: str = "openai:gpt-4o", return_messages: bool = False):
    """
    Executes a research task using tools via aisuite (no manual loop).
    Returns either the assistant text, or (text, messages) if return_messages=True.
    """
    print("==================================")  
    print("🔍 Research Agent")                 
    print("==================================")

    current_time = datetime.now().strftime('%Y-%m-%d')
    
    ### START CODE HERE ###

    prompt = f"""
    You are a research assistant with access to the following tools:
    - arxiv_tool: Use this to find academic papers on arXiv.
    - tavily_tool: Use this for general web searches.
    - wikipedia_tool: Use this for accessing encyclopedic knowledge.

    Current date: {current_time}

    Your task: {task}
    """
    
    messages = [{"role": "user", "content": prompt}]

    tools = [
        research_tools.arxiv_search_tool,
        research_tools.tavily_search_tool,
        research_tools.wikipedia_search_tool
    ]
    
    response = CLIENT.chat.completions.create(  
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_turns=6
    )  
    
    ### END CODE HERE ###

    content = response.choices[0].message.content
    print("✅ Output:\n", content)

    
    return (content, messages) if return_messages else content

In [ ]:
# Test your code!
unittests.test_research_agent(research_agent)

## Exercise 3: writer_agent

### Objective
Set up a call to a language model (LLM) for executing writing tasks like drafting, expanding, or summarizing text.

### Instructions

1. **Focus Areas**:
   - **System Prompt**:
     - Define `system_prompt` to assign the LLM the role of a writing agent focused on generating academic or technical content.
   - **System and User Messages**:
     - Create `system_msg` using `{"role": "system", "content": system_prompt}`.
     - Create `user_msg` using `{"role": "user", "content": task}`.
   - **Messages List**:
     - Combine `system_msg` and `user_msg` into a `messages` list.

In [ ]:
# GRADED FUNCTION: writer_agent
def writer_agent(task: str, model: str = "openai:gpt-4o") -> str:
    """
    Executes writing tasks, such as drafting, expanding, or summarizing text.
    """
    print("==================================")
    print("✍️ Writer Agent")
    print("==================================")

    ### START CODE HERE ###
    
    system_prompt = "You are a writing agent specialized in generating well-structured academic or technical content."

    system_msg = {"role": "system", "content": system_prompt}

    user_msg = {"role": "user", "content": task}

    messages = [system_msg, user_msg]
    
    ### END CODE HERE ###

    response = CLIENT.chat.completions.create(
        model=model, 
        messages=messages,
        temperature=1.0
    )

    return response.choices[0].message.content

In [ ]:
# Test your code!
unittests.test_writer_agent(writer_agent)

## Exercise 4: editor_agent

### Objective
Configure a call to a language model (LLM) to perform editorial tasks such as reflecting, critiquing, or revising drafts.

### Instructions

1. **Focus Areas**:
   - **System Prompt**:
     - Define `system_prompt` to assign the LLM the role of an editor agent whose task is to reflect on, critique, or improve drafts.
   - **System and User Messages**:
     - Create `system_msg` using `{"role": "system", "content": system_prompt}`.
     - Create `user_msg` using `{"role": "user", "content": task}`.
   - **Messages List**:
     - Combine `system_msg` and `user_msg` into a `messages` list.

In [ ]:
# GRADED FUNCTION: editor_agent
def editor_agent(task: str, model: str = "openai:gpt-4o") -> str:
    """
    Executes editorial tasks such as reflection, critique, or revision.
    """
    print("==================================")
    print("🧠 Editor Agent")
    print("==================================")
    
    ### START CODE HERE ###

    system_prompt = "You are an editor agent specialized in reflecting on, critiquing, or improving existing drafts."
    
    system_msg = {"role": "system", "content": system_prompt}
    
    user_msg = {"role": "user", "content": task}
    
    messages = [system_msg, user_msg]
    
    ### END CODE HERE ###
    
    response = CLIENT.chat.completions.create(
        model=model, 
        messages=messages,
        temperature=0.7 
    )
    
    return response.choices[0].message.content

In [ ]:
# Test your code!
unittests.test_editor_agent(editor_agent)

### 🎯 The Executor Agent

The `executor_agent` manages the workflow by executing each step of a given plan. It:

1. Decides **which agent** (`research_agent`, `writer_agent`, or `editor_agent`) should handle the step.
2. Builds context from the outputs of previous steps.
3. Sends the enriched task to the selected agent.
4. Collects and stores the results in a shared history.

👉 **Do not implement or modify this function.** It is already provided as the orchestration component of the multi-agent pipeline.

Notice that `planner_agent` might return a long list of steps. Because of this, the maximum number of steps is set to a maximum of 4 to keep running time reasonable.

In [ ]:
agent_registry = {
    "research_agent": research_agent,
    "editor_agent": editor_agent,
    "writer_agent": writer_agent,
}

def clean_json_block(raw: str) -> str:
    """
    Clean the contents of a JSON block that may come wrapped with Markdown backticks.
    """
    raw = raw.strip()
    if raw.startswith("```"):
        raw = re.sub(r"^```(?:json)?\n?", "", raw)
        raw = re.sub(r"\n?```$", "", raw)
    return raw.strip()

def executor_agent(topic, model: str = "openai:gpt-4o", limit_steps: bool = True):

    plan_steps = planner_agent(topic)
    max_steps = 4

    if limit_steps:
        plan_steps = plan_steps[:min(len(plan_steps), max_steps)]
    
    history = []

    print("==================================")
    print("🎯 Editor Agent")
    print("==================================")

    for i, step in enumerate(plan_steps):

        agent_decision_prompt = f"""
        You are an execution manager for a multi-agent research team.

        Given the following instruction, identify which agent should perform it and extract the clean task.

        Return only a valid JSON object with two keys:
        - "agent": one of ["research_agent", "editor_agent", "writer_agent"]
        - "task": a string with the instruction that the agent should follow

        Only respond with a valid JSON object. Do not include explanations or markdown formatting.

        Instruction: "{step}"
        """
        response = CLIENT.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": agent_decision_prompt}],
            temperature=0,
        )

        raw_content = response.choices[0].message.content
        cleaned_json = clean_json_block(raw_content)
        agent_info = json.loads(cleaned_json)

        agent_name = agent_info["agent"]
        task = agent_info["task"]

        context = "\n".join([
            f"Step {j+1} executed by {a}:\n{r}" 
            for j, (s, a, r) in enumerate(history)
        ])
        enriched_task = f"""
        You are {agent_name}.

        Here is the context of what has been done so far:
        {context}

        Your next task is:
        {task}
        """

        print(f"\n🛠️ Executing with agent: `{agent_name}` on task: {task}")

        if agent_name in agent_registry:
            output = agent_registry[agent_name](enriched_task)
            history.append((step, agent_name, output))
        else:
            output = f"⚠️ Unknown agent: {agent_name}"
            history.append((step, agent_name, output))

        print(f"✅ Output:\n{output}")

    return history

In [ ]:
# If you want to see the full workflow without limiting the number of steps. Set limit_steps to False
# Keep in mind this could take more than 10 minutes to complete
executor_history = executor_agent("The ensemble Kalman filter for time series forecasting", limit_steps=True)

In [ ]:
md = executor_history[-1][-1].strip("`")  
display(Markdown(md))